# logit/probit/LPM-регрессия: прогнозирование

In [32]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

# Не показывать FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## labour force equation

Для датасета [mroz_Green](http://www.stern.nyu.edu/~wgreene/Text/Edition7/TableF5-1.csv)
рассморим регрессию **LFP на WA, log(FAMINC), WE, KL6, K618, CIT, UN**
трёх спецификаций:

- LPM
- logit
- probit

In [42]:
# подключим датасет mroz_Greene по ссылке 
df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/econometrica/main/econometrica2/data-csv/TableF5-1.csv', na_values=(' ', '', '  '))
#подключим датасет mroz_Greene из локального файла
#df = pd.read_csv('TableF5-1.csv', na_values=(' ', '', '  '))

## Спецификация и подгонка

In [34]:
# LPM 
res_lpm_hc = smf.ols(formula='LFP~WA+np.log(FAMINC)+WE+KL6+K618+CIT+UN', data=df).fit(cov_type='HC3')

In [35]:
# logit
res_logit = smf.logit(formula='LFP~WA+np.log(FAMINC)+WE+KL6+K618+CIT+UN', data=df).fit()

Optimization terminated successfully.
         Current function value: 0.614104
         Iterations 5


In [36]:
# probit
res_probit = smf.probit(formula='LFP~WA+np.log(FAMINC)+WE+KL6+K618+CIT+UN', data=df).fit()

Optimization terminated successfully.
         Current function value: 0.614282
         Iterations 5


In [37]:
# Сравнение моделей
print(summary_col([res_lpm_hc, res_logit, res_probit], model_names=['Robust LPM', 'logit', 'probit'], float_format="%.3f", stars=True, info_dict = {'Observations': lambda x: '{0:d}'.format(int(x.nobs)), 'Log Likelihood': lambda x: '{:.3f}'.format(x.llf), 'Akaike Inf. Crit.': lambda x: '{:.3f}'.format(x.aic)}))


                  Robust LPM   logit     probit 
------------------------------------------------
CIT               -0.048     -0.214    -0.126   
                  (0.037)    (0.176)   (0.107)  
Intercept         0.079      -1.856    -1.108   
                  (0.362)    (1.679)   (1.014)  
K618              -0.018     -0.095    -0.057   
                  (0.014)    (0.067)   (0.040)  
KL6               -0.302***  -1.443*** -0.868***
                  (0.034)    (0.194)   (0.112)  
R-squared         0.130                         
R-squared Adj.    0.122                         
UN                -0.004     -0.017    -0.011   
                  (0.006)    (0.026)   (0.016)  
WA                -0.013***  -0.063*** -0.038***
                  (0.002)    (0.013)   (0.008)  
WE                0.038***   0.179***  0.108*** 
                  (0.008)    (0.040)   (0.024)  
np.log(FAMINC)    0.075**    0.341**   0.205**  
                  (0.038)    (0.172)   (0.104)  
Observations      7

## Прогнозирование

In [38]:
# Создадим датафрейм с новыми данными регрессоров для прогноза
new_data = pd.DataFrame({'WA':[35, 40, 42], 
                         'FAMINC':[12500, 9800, 67800], 
                         'WE': [15, 12, 14], 
                         'KL6': [2, 1, 2],
                         'K618': [0, 2, 1], 
                         'CIT': [1, 0, 1], 
                         'UN':[5, 7.5, 3]})
new_data

,WA,FAMINC,WE,KL6,K618,CIT,UN
0,35,12500,15,2,0,1,5.0
1,40,9800,12,1,2,0,7.5
2,42,67800,14,2,1,1,3.0


In [39]:
# Прогноз для LPM с округлением до 3-х десятичных знаков
res_lpm_hc.predict(exog=new_data, transform=True).round(3)

0    0.221
1    0.329
2    0.207
dtype: float64

In [40]:
# Прогноз для logit с округлением до 3-х десятичных знаков
res_logit.predict(exog=new_data, transform=True).round(3)

0    0.209
1    0.300
2    0.192
dtype: float64

In [41]:
# Прогноз для probit с округлением до 3-х десятичных знаков
res_probit.predict(exog=new_data, transform=True).round(3)

0    0.214
1    0.307
2    0.196
dtype: float64